In [43]:
# Do all package imports

import neurokit2 as nk
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
from pathlib import Path
import math

In [44]:
file_path = 'C:/Users/eleon/Desktop/SDAP/backend/src/data/p1_wk1/1222325cFnorm.csv'

In [45]:
ecg = pl.read_csv(file_path, columns=['ECG'])

In [46]:
ecg_clean = nk.ecg_clean(ecg, sampling_rate=2000)

In [47]:
ecg_peaks = nk.ecg_findpeaks(ecg_clean, sampling_rate=2000)

In [48]:
ecg_clean_peaks = nk.signal_fixpeaks(ecg_peaks, sampling_rate=2000, iterative=False, show=False, method="Kubios")

In [49]:
ecg_clean_peaks = ecg_clean_peaks[1]

In [53]:
def compute_HRV(peaks):
    duration_peaks = peaks[len(peaks)-1]
    divider = duration_peaks/2000/60/5
    segment = np.array_split(peaks, math.ceil(divider))

    hrv_segment_df=pl.DataFrame()

    for i in range(len(segment)):
        hrv_segment=nk.hrv(segment[i],sampling_rate=2000, show=False)
        hrv_segment_df = pl.concat([hrv_segment_df,hrv_segment], rechunk=True)

    return hrv_segment_df

    

In [54]:
hrv = compute_HRV(ecg_clean_peaks)

AttributeError: 'DataFrame' object has no attribute '_df'

In [52]:
hrv

,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,925.654639,431.503201,111.426208,443.553578,NaN,NaN,NaN,NaN,518.765893,519.662508,...,0.486538,7.885839,0.740774,1.221496,1.104392,1.091583,0.923459,1.902239,2.907596,0.787548
1,847.197595,142.506635,93.380590,106.546967,NaN,NaN,NaN,NaN,87.934311,88.083740,...,1.032083,7.846287,0.809766,0.703711,0.998157,1.441702,1.530256,1.735085,2.616988,0.337521
2,917.501718,125.013989,23.000457,91.894706,NaN,NaN,NaN,NaN,126.235952,126.453404,...,0.905935,7.643470,0.817308,0.526867,1.162670,1.161958,0.973499,1.868938,1.825529,0.731295
3,915.168966,142.881411,39.683697,117.929410,NaN,NaN,NaN,NaN,175.373613,175.676550,...,1.163594,7.706515,0.831401,0.994140,1.252796,1.439747,0.943630,1.808409,1.705754,0.846197
4,919.272414,106.039882,54.170056,88.483347,NaN,NaN,NaN,NaN,78.503322,78.639417,...,1.378197,7.703568,1.023082,1.128218,1.382338,1.715275,1.361733,1.796186,2.164565,0.959024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,1032.184483,124.183310,45.536902,112.608264,NaN,NaN,NaN,NaN,65.762213,65.875983,...,1.345511,7.851343,0.932284,1.346145,1.309004,1.949874,1.772802,1.529405,2.680029,0.789784
114,924.308621,112.113359,48.215065,92.424200,NaN,NaN,NaN,NaN,46.353342,46.432388,...,1.048488,7.782377,0.749331,1.004685,1.171882,1.654438,1.613368,1.570980,2.211493,0.564132
115,929.606897,121.647780,69.240569,104.743780,NaN,NaN,NaN,NaN,62.522411,62.627892,...,1.026157,7.808273,0.742329,1.374547,1.296544,1.851577,1.617523,1.556102,2.666952,0.705165
116,905.927586,169.762749,104.090182,120.516583,NaN,NaN,NaN,NaN,51.327979,51.410588,...,0.876727,7.974703,0.621603,1.464344,1.337302,1.782882,1.506085,1.394867,2.275246,0.423099


In [40]:
def get_coordinates(df, x_max=18):
    # Add x coordinate
    df = df.with_columns(
        (pl.arange(0, df.height()) % x_max).alias('x')
    )

    # Add y coordinate
    df = df.with_columns(
        (pl.arange(0, df.height()) // x_max).alias('y')
    )

    return df

In [42]:
get_coordinates(hrv)

AttributeError: 'DataFrame' object has no attribute 'with_columns'